In [16]:
import numpy as np
import pandas as pd
import time
import pickle

In [14]:
#Load landarea dataset
LandAreaFileLoc = "/home/henry/Insight/Yogee/Datasets/NY_Land_Area/NY_Land_Area.csv"
LandAreaDf = pd.read_csv(LandAreaFileLoc)

In [15]:
#Get zipcode, latitude and longitude data
NYZipcodeDistanceDf = LandAreaDf.loc[:,['GEO.id2','Latitude','Longitude']]


In [36]:
import pickle

f = open('/home/henry/Downloads/YogaMap/ZipcodeDistanceDf.pckl', 'wb')
pickle.dump(ZipcodeDistanceDf, f)
f.close()

In [22]:
from math import radians, cos, sin, asin, sqrt
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    # Radius of earth in kilometers is 6371
    km = 6371* c
    mile = 0.621371 * km
    return mile

In [ ]:
from math import radians, cos, sin, asin, sqrt
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    # Radius of earth in kilometers is 6371
    km = 6371* c
    mile = 0.621371 * km
    return mile

In [124]:
def closestzipcode(Lon1,Lat1,ZipcodeDistanceDf):
    LonArray = np.float64(ZipcodeDistanceDf.loc[:,'Longitude'])
    LatArray = np.float64(ZipcodeDistanceDf.loc[:,'Latitude'])
    Lon1Radian = np.ones(np.size(LonArray))*(Lon1*(np.pi/180))
    Lat1Radian = np.ones(np.size(LatArray))*(Lat1*(np.pi/180))
    LonRadian = LonArray*(np.pi/180)
    LatRadian = LatArray*(np.pi/180)
    # haversine formula 
    DiffLon = LonRadian - Lon1Radian
    DiffLat = LatRadian - Lat1Radian
    a = np.square(np.sin(DiffLat/2)) + np.multiply(np.multiply(np.cos(Lat1Radian),np.cos(LatRadian)), np.square(np.sin(DiffLon/2)))
    c = 2 * np.arcsin(np.sqrt(a))
    # Radius of earth in kilometers is 6371
    km = 6371* c
    mile = 0.621371 * km
    DistanceDF = pd.DataFrame(mile,columns=['Distance'])
    ZipcodeDistanceDf = pd.concat([ZipcodeDistanceDf, DistanceDF], axis=1, sort=False)
    ZipcodeDistanceDf = ZipcodeDistanceDf.sort_values('Distance', ascending=True)
    return ZipcodeDistanceDf


In [120]:
def closestzipcode(lon,lat,ZipcodeDistanceDf):
    NanDfValues = np.zeros([np.shape(ZipcodeDistanceDf)[0],1])
    NanDfValues[:] = np.nan
    NanDf = pd.DataFrame(NanDfValues,columns=['Distance'])
    ZipcodeDistanceDf = pd.concat([ZipcodeDistanceDf, NanDf], axis=1, sort=False)
    for i in range(0,np.shape(ZipcodeDistanceDf)[0]):
        ziplon = ZipcodeDistanceDf.loc[i,'Longitude']
        ziplat = ZipcodeDistanceDf.loc[i,'Latitude']
        distance = haversine(lon, lat, ziplon, ziplat)
        ZipcodeDistanceDf.loc[i,'Distance'] = distance
        ZipcodeDistanceDf = ZipcodeDistanceDf.sort_values('Distance', ascending=True)
    return ZipcodeDistanceDf


In [125]:
lon = -73.9972808
lat = 40.750672
NewZipcodeDistanceDf = closestzipcode(lon,lat,NYZipcodeDistanceDf)
NewZipcodeDistanceDf.head(10)

,GEO.id2,Latitude,Longitude,Distance
1,10001,40.750672,-73.997281,4.395101e-13
64,10199,40.751393,-73.997229,4.986269e-02
48,10119,40.750310,-73.992979,2.265629e-01
16,10018,40.755329,-73.993127,3.883493e-01
10,10011,40.742002,-74.000594,6.236593e-01
34,10036,40.759280,-73.989909,7.089540e-01
44,10110,40.754499,-73.982256,8.296428e-01
61,10173,40.754131,-73.979364,9.677624e-01
54,10165,40.752131,-73.978722,9.765976e-01
56,10168,40.751448,-73.977103,1.057494e+00


In [47]:
f = open('/home/henry/Insight/Yogee/Model/PredictDF.pckl', 'rb')
PredictDf = pickle.load(f)
f.close()

In [45]:
InputZipcode = '10002'
InputZipcode = np.int64(InputZipcode)
InputZipRow = NYZipcodeDistanceDf[NYZipcodeDistanceDf['GEO.id2']==InputZipcode]
InputLon = InputZipRow['Longitude'].iloc[0]
InputLat = InputZipRow['Latitude'].iloc[0]
CloseZipcodeDf = closestzipcode(InputLon,InputLat,NYZipcodeDistanceDf)
CloseZipcodeDf = CloseZipcodeDf[CloseZipcodeDf['Distance']<5]

In [74]:
NumZipRecommend = 0
i = 0
ZipRecommend = []
ScoreRecomend = []
LonRecommend = []
LatRecommend = []    

while (NumZipRecommend < 5) and (i < CloseZipcodeDf.shape[0]): 
    ZipRow = CloseZipcodeDf.iloc[i,0]
    LonRow = CloseZipcodeDf.iloc[i,2]
    LatRow = CloseZipcodeDf.iloc[i,1]
    PredictRow = PredictDf[PredictDf['zip']==ZipRow]
    if not PredictRow.empty:
        Score = PredictRow['Score'].iloc[0]
        if Score > 5:
            ZipRecommend.append(ZipRow)
            ScoreRecomend.append(PredictRow['Score'])
            LatRecommend.append(LatRow)
            LonRecommend.append(LonRow)
            NumZipRecommend = NumZipRecommend + 1            
    i = i + 1

In [78]:
ZipRecommend

[10002, 10009, 10012, 10038, 10013]